<a href="https://colab.research.google.com/github/Biocanter/RNA_Colab_Code/blob/main/RNA_secondary_structure_parameters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The goal of this notebook is to calculate some RNA secondary structure parameters: minimum free energy (MFE) structure, Sum Unpaired Probability (SUP), etc. 

https://daslab.stanford.edu/site_data/pub_pdf/LeppekEtAl_NatureCommunications_2022.pdf




### **1.- Install RNA secondary structure software**
https://github.com/Biocanter/RNA_Colab_Code/blob/main/arnie(ViennaRNA%2CContraFold%2CEternaFold).ipynb

In [ ]:
###Install Contrafold
!git clone https://github.com/csfoo/contrafold-se.git
!apt-get install -y g++-4.8
!sed -i.bak "1 s/^.*$/CXX = g++-4.8/" contrafold-se/src/Makefile
!cd contrafold-se/src; make
##Install arnie
!git clone https://github.com/DasLab/arnie
##Install EternaFold
!git clone https://github.com/eternagame/EternaFold
!cd /content/EternaFold/src; make
##Instal Conda
! wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
! chmod +x Miniconda3-latest-Linux-x86_64.sh
! bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
###Install ViennaRNA
!conda install -c bioconda viennarna -y
###set_up
import os
import sys
!echo "vienna_2: /usr/local/bin" > arnie.conf
!echo "eternafold: /content/EternaFold/src" >> arnie.conf
!echo "contrafold_2: /content/contrafold-se/src" >> arnie.conf
!echo "eternafoldparams: /content/EternaFold/parameters/EternaFoldParams.v1" >> arnie.conf
!echo "TMP: /content/tmp" >> arnie.conf
!mkdir -p /content/tmp
os.environ["ARNIEFILE"] = f"arnie.conf"
%load_ext autoreload
%autoreload 2
%pylab inline
##arnie libraries
from arnie.pfunc import pfunc
from arnie.free_energy import free_energy
from arnie.bpps import bpps
from arnie.mfe import mfe
import arnie.utils as utils
from decimal import Decimal
from arnie.mea.mea import MEA
##Python libraries
import seaborn as sns
sns.set_context('poster')
sns.set_style('white')
import pandas as pd
import numpy as np

### **2.- RNA secondary structure parameters**
Code from .-OpenVaccine-solves.-https://eternagame.org/software

eternagame-OpenVaccine-solves/scripts/calculate_metrics.py

In [2]:
def get_mean_bp_prox(bp_matrix):
  N=bp_matrix.shape[0]
  ii, jj = np.meshgrid([x for x in range(N)], [x for x in range(N)])
  inds = np.clip(ii-jj,0,1e12)
  matrix = np.multiply(inds, bp_matrix)
  bp_mat_norm = bp_matrix[np.where(inds>0)]
  return(np.sum(matrix)/np.sum(bp_mat_norm))
def fold_RNA(seq, pacakge):
  seq=seq['RNA_sequence']
  #MFE
  mfe_struct = mfe(seq, package=package)
  dG_MFE_struct = free_energy(seq, package=package,constraint=mfe_struct.replace('.','x'), linear=False)
  ##nt unpaired
  nt_unpaired=mfe_struct.count('.')
  ### AUP and SUP
  bp_matrix = bpps(seq, package=package)
  p_unp_vec = 1 - np.sum(bp_matrix, axis=0)
  SUP=np.sum(p_unp_vec)
  AUP=np.mean(p_unp_vec)
  #mean_bp_prox_Vienna_CDS
  mean_bp_prox_Vienna_CDS=get_mean_bp_prox(bp_matrix)
  ###dG_ensemble
  dG_ensemble = free_energy(seq, package=package,linear=False)
  ###GC content
  return (mfe_struct,dG_MFE_struct,p_unp_vec,mean_bp_prox_Vienna_CDS,AUP, SUP,dG_ensemble,nt_unpaired)

### **3.- Generate pandas Dataframe with some HCV RNA sequences**

In [3]:
#create list with RNA information
Rfam_ID=['IRES_HCV','HepC_CRE','HCV_ARF_SL','HCV_SLVII','HCV_X3']
Rfam_accesion=['RF00061','RF00260','RF00620','RF00468','RF00481']
RNA_sequence=['GAGUGUUGUACAGCCUCCAGGACCCCCCCUCCCGGGAGAGCCAUAGUGGUCUGCGGAACCGGUGAGUACACCGGAAUUGCCGGGAUGACCGGGUCCUUUCUUGGAUUAAACCCGCUCAAUGCCCGGAGAUUUGGGCGUGCCCCCGCAAGACUGCUAGCCGAGUAGCGUUGGGUUGCGAAAGGCCUUGUGGU',
              'UACAGCGGGGGAGACAUUUAUCACAGCGUGUCUCAUGCCCGGCCCCGCUGG',
              'AGAAAAACCAAACGUAACACCAGCCGUCGCCCACAGGACGUCAAGUUCCCGGGCGGCGGCCAGAUCGUUGGUGGAGUUUACUUGUUGCCGCGCAGGGGCCCUAGAUUGGGUGUGCGCGCGACGAGGAAGACUUCCGAGCGGUCGCAACCUCGA',
              'AAGCUCCUGUCCCAGGGGGGGAGGGCUGCCAACUGUGGCAAAUACCUCUUCAACUGGGCAGUAAGG',
              'CUGGUGGCUCCAUCUUAGCCCUAGUCACGGCUAGCUGUGAAAGGUCCGUGAGCCGCAUGACUGCAGAGAUUGCCGUAACUGGUAUCUCUGCAGAUCAUGU']
###zip list
df = pd.DataFrame(list(zip(Rfam_ID, Rfam_accesion,RNA_sequence)),columns =['Rfam_ID', 'Rfam_accesion','RNA_sequence'])

In [4]:
##calculate parameters
package=['eternafold','vienna_2']
for package in package:
  df[[f'MFE Struct {package}', 
  f'dG(MFE) {package}',
  f'punp_vec {package}',
  f'mean bp prox {package}', 
  f'AUP {package}', 
  f'SUP {package}',
  f'dG(ensemble) {package}',
  f'n_unpaired_nts_MFE {package}']]=df.apply(lambda x: fold_RNA(x, package), axis=1, result_type='expand')

In [5]:
df

,Rfam_ID,Rfam_accesion,RNA_sequence,MFE Struct eternafold,dG(MFE) eternafold,punp_vec eternafold,mean bp prox eternafold,AUP eternafold,SUP eternafold,dG(ensemble) eternafold,n_unpaired_nts_MFE eternafold,MFE Struct vienna_2,dG(MFE) vienna_2,punp_vec vienna_2,mean bp prox vienna_2,AUP vienna_2,SUP vienna_2,dG(ensemble) vienna_2,n_unpaired_nts_MFE vienna_2
0,IRES_HCV,RF00061,GAGUGUUGUACAGCCUCCAGGACCCCCCCUCCCGGGAGAGCCAUAG...,..........(.((((...((((((...(((.(((.....(..(((...,-38.4000,"[0.9005189892000001, 0.901385941, 0.8430863531...",64.679817,0.461506,88.147650,-58.6764,71,..............((((.(((.......)))..)))).(((((((...,-74.5,"[0.7550315611873246, 0.765221026439203, 0.6128...",57.488255,0.357610,68.303525,-77.28,67
1,HepC_CRE,RF00260,UACAGCGGGGGAGACAUUUAUCACAGCGUGUCUCAUGCCCGGCCCC...,..(((((((((((((((..........)))))))........))))...,-18.0842,"[0.974049369, 0.999710284, 0.07593927280000001...",29.284072,0.433573,22.112201,-19.1942,21,..(((((((((((((((..........)))))))........))))...,-23.0,"[0.9991819419363113, 0.9999961543809494, 0.004...",28.689679,0.398491,20.323017,-23.38,21
2,HCV_ARF_SL,RF00620,AGAAAAACCAAACGUAACACCAGCCGUCGCCCACAGGACGUCAAGU...,......................((((((((((...((((.....))...,-45.9893,"[0.995917226, 0.9547266056, 0.9936930776, 0.99...",35.837520,0.445524,68.165135,-52.4911,61,......................((((((((((...((((.....))...,-70.8,"[0.9957211532091144, 0.9385910213154726, 0.974...",37.800603,0.397989,60.892250,-72.41,61
3,HCV_SLVII,RF00468,AAGCUCCUGUCCCAGGGGGGGAGGGCUGCCAACUGUGGCAAAUACC...,...((.(((.(((((..(((((((..(((((....)))))....))...,-21.4290,"[0.99929814, 0.9989294564, 0.9877893303, 0.605...",32.338393,0.416670,27.500221,-25.0190,22,......(((.(((((..(((((((..(((((....)))))....))...,-31.8,"[0.9998668510699421, 0.9996280816841728, 0.996...",31.620952,0.382338,25.234320,-32.38,26
4,HCV_X3,RF00481,CUGGUGGCUCCAUCUUAGCCCUAGUCACGGCUAGCUGUGAAAGGUC...,((((.((((.......))))))))(((((((...(.......)).)...,-24.9661,"[0.6558412945000001, 0.6377399467, 0.578792354...",23.884253,0.362787,36.278745,-35.1464,28,((((.((((.......))))))))((((((...(((......))))...,-44.5,"[0.41157513814862323, 0.39147733961914555, 0.3...",23.006539,0.293610,29.360973,-46.58,28
